In [47]:
from langchain_groq import ChatGroq
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain.agents import initialize_agent,AgentType
from langchain_google_community import GoogleSearchAPIWrapper
from langchain_core.tools import Tool
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage

import os
from dotenv import load_dotenv
load_dotenv()

True

In [48]:
GROG_API_KEY = os.getenv("GROQ_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["GOOGLE_CSE_ID"] = GOOGLE_CSE_ID
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "SearchEngineApp"

In [49]:
search = GoogleSearchAPIWrapper()

google_tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func= lambda query: search.results(query, 1),
)

In [50]:
llm=ChatGroq(groq_api_key=GROG_API_KEY,model_name="Llama3-70b-8192",streaming=True)
tools=[google_tool]

search_agent=initialize_agent(tools,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, handling_parsing_errors=True)


In [51]:

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(search_agent,get_session_history)

In [52]:
config={"configurable":{"session_id":"chat1"}}

In [55]:
llm.invoke('Hello ? ')

AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?", response_metadata={'finish_reason': 'stop'}, id='run-afe12539-0624-4696-80cc-20eb76e4c6ce-0', usage_metadata={'input_tokens': 13, 'output_tokens': 26, 'total_tokens': 39})

In [53]:
question = HumanMessage(content='Who is Abdellah Kaissari ?')
response = with_message_history.invoke(question, config=config)
print(response)

{'input': [HumanMessage(content='Who is Abdellah Kaissari ?')], 'output': 'Unfortunately, I was unable to find any information about who Abdellah Kaissari is.'}


In [54]:
with_message_history.invoke(
    [HumanMessage(content="What was my question about ?")],
    config=config,
)

KeyboardInterrupt: 